In [1]:
import chromadb
from openai.embeddings_utils import get_embedding
from dotenv import load_dotenv
from chromadb.utils import embedding_functions
import os
from chromadb.config import Settings

In [2]:
load_dotenv()
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")
EMBEDDINGS_MODEL = "text-embedding-ada-002"

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key=OPEN_API_KEY,
                model_name=EMBEDDINGS_MODEL
            )
CHROMA_CLIENT = chromadb.Client(Settings(
        chroma_db_impl="duckdb+parquet", persist_directory="../chromadb"))
CHROMA_CLIENT.persist()
collection = CHROMA_CLIENT.get_or_create_collection(name="teli-ai",embedding_function=openai_ef)

Using embedded DuckDB with persistence: data will be stored in: ../chromadb


In [3]:
collection.count()

31

In [6]:
response = collection.query(query_texts=["Customer: I know what you guys do…You guys bullshit an offer that is very low and I am not going to fall for that\n"],n_results=3)
suggestions = ""
for doc in response["documents"]:
    for line in doc:
        suggestions += line + "\n\n"

print(suggestions)


Customer: I know what you guys doâ€¦You guys bullshit an offer that is very low and I am not going to fall for that
AI: Thats not at all what we do. We will literally pay you all cash for what the property is worth and that depends on if it needs work or not

Customer: Im sorry but I am not interested
AI: Okay

Customer: I have never owned a property in my life so I don't know what you are talking about
AI: Okay




In [ ]:
print(response)

In [ ]:
from transformers import GPT2Tokenizer

def count_gpt_tokens(passage):
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    tokens = tokenizer.encode(passage, add_special_tokens=True)
    token_count = len(tokens)
    return token_count


In [ ]:
prompt = "Act as a telemarketer. Your name is Amelio from Pure Investment. Follow this process:\nYou initiated the call to customer. First, you need to ask customer question to make sure you are talking to the right customer. Then, you need to ask if he/she is living on the right adress.If the information is correct, ask if he/she is interested in selling their current property for cash.\nYou are talking to {customer_name}, living in {customer_address}.\nOnly when the customer is interested in selling their property, ask them for the property condition. Then, ask if they would sell it for {price}.\nOtherwise, if customer is not interested or it is the wrong cusomter, politely end the conversation and hang up.\nYou only address the customer's previous question and avoid providing extra information or generating unrelated sequences.\n\nSample conversation flow:\n{sample_conversation}\n\nCurrent conversation:\nYou : [initiate call]\n".format(customer_name="Vinny", customer_address="Vinny", price="123123",sample_conversation="sample_conversation")
token_count = count_gpt_tokens(prompt)
print("Number of GPT tokens:", token_count)